In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#Instantiate the CRUD program
from CRUD import AnimalShelter




###########################
# Data Manipulation / Model
###########################
#Hardcode the username and password and use them in the AnimalShelter call
username = "ZaneBrown"
password = "Zoe"
shelter = AnimalShelter(username, password)


# reads the data from our csv file 
#df = pd.DataFrame.from_records(shelter.read({}))
df = pd.read_csv("aac_shelter_outcomes.csv")



#########################
# Dashboard Layout / View
#########################

#Initiate the app
app = JupyterDash('SimpleExample')

#Set up the variables needed to implement the image
image_filename = 'Grazioso-Salvare-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#Initiate the app layout with title and image
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Zane Brown SNHU CS-340 Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Hr(),
    
    #Added in code for the interactive filtering option with buttons.
    
    html.Div(className='row',
            style={'display': 'flex'},
                 children=[
                     html.Button(id='btn-1', n_clicks_timestamp='0', children='Water Rescue'),
                     html.Button(id='btn-2', n_clicks_timestamp='0', children='Mountain Rescue'),
                     html.Button(id='btn-3', n_clicks_timestamp='0', children='Disaster Rescue'),
                     html.Button(id='btn-4', n_clicks_timestamp='0', children='Reset'),
                     html.Div(id='container-button-timestamp')
                 ]),

    
    #Add in the data table that houses our csv data
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Set up the features for your interactive data table to make it user-friendly for your client
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,

    ),
    
    html.Br(),
    html.Hr(),
    
    #ADDING PIE CHART HERE
    html.Div([
        html.Label(['Animal Shelter Pie Chart']),
        dcc.Dropdown(
            id='my_dropdown',
            options=[
                    {'label': 'Animal Type', 'value': 'animal_type'},
                    {'label': 'Breed', 'value': 'breed'},
                    {'label': 'Age', 'value': 'age_upon_outcome'},
                    {'label': 'Color', 'value': 'color'}
            ],
            value='animal_type',
            multi=False,
            clearable=False,
            style={"width": "50%"}
        ),
    ]),
    
    
    html.Div([
        dcc.Graph(id='the_graph')
    ]),
    
     html.Br(),
     html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                 dcc.Graph(id="graph"),
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    
    html.Br(),
    html.Hr(),
    html.Div(id='bar-container'),
    html.Div(id='choromap-container'),
    
    
    
])

#############################################
# Interaction Between Components / Controller
#############################################

#----------------------------------------------------------------------------------
#DATA TABLE BUTTON CALLBACK
@app.callback(Output('datatable-interactivity', 'data'),
             [Input('btn-1', 'n_clicks_timestamp'),
              Input('btn-2', 'n_clicks_timestamp'),
              Input('btn-3', 'n_clicks_timestamp'),
              Input('btn-4', 'n_clicks_timestamp')])
def on_click(btn1, btn2, btn3, btn4):
    if int(btn1) > int(btn2) and int(btn1) > int(btn3) and int(btn1) > int(btn4):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix",
                                                "Chesapeake Bay Retriever","Newfoundland"]}, 
                                                "sex_upon_outcome":"Intact Female",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$It":156}})))
    elif int(btn2) > int(btn1) and int(btn2) > int(btn3) and int(btn2) > int(btn4):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog","breed":{"$in":["German Shepherd Alaskan",
                                                "Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]}, 
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":26},
                                                "age_upon_outcome_in_weeks":{"$It":156}})))
    elif int(btn3) > int(btn1) and int(btn3) > int(btn2) and int(btn3) > int(btn4):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher",
                                                "German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]}, 
                                                "sex_upon_outcome":"Intact Male",
                                                "age_upon_outcome_in_weeks":{"$gte":20},
                                                "age_upon_outcome_in_weeks":{"$It":300}})))
    elif int(btn4) > int(btn1) and int(btn4) > int(btn2) and int(btn4) > int(btn3):
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
    return df.to_dict('records')
#----------------------------------------------------------------------------------


#----------------------------------------------------------------------------------
#UPDATE STYLES CALLBACK

# DOESN'T WORK __ Causes the entire program to hang. Attempted to fix it multiple times. continues to hang program


# @app.callback(
#    Output('datatable-id', 'style_data_conditional'),
#    [Input('datatable-id', 'selected_columns')]
# )
# def update_styles(selected_columns):
#     return [{
#        'if': { 'column_id': i },
#        'background_color': '#D2F3FF'
#    } for i in selected_columns]

#----------------------------------------------------------------------------------



#----------------------------------------------------------------------------------
#PIE CHART CALLBACK
@app.callback(
    Output(component_id='the_graph', component_property='figure'),
    [Input(component_id='my_dropdown', component_property='value')]
)

def update_graph(my_dropdown):
    dff = df
    
    piechart=px.pie(
            data_frame=dff,
            names=my_dropdown,
            hole=.3,
            )
    
    return (piechart)
#----------------------------------------------------------------------------------


#----------------------------------------------------------------------------------
#BAR CHART CALLBACK
@app.callback(
    Output(component_id='bar-container', component_property='children'),
    [Input(component_id='datatable-interactivity', component_property="derived_virtual_data"),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_rows')]
)
def update_bar(all_rows_data, slctd_row_indices):
    print('*************************************************************************')
    print('Data across all pages pre or post filtering: {}'.format(all_rows_data))
    print('----------------------------------------------')
    print("Indices of selected rows if part of table after filtering:{}".format(slctd_row_indices))
    print("Names of selected rows if part of table after filtering: {}".format(slct_rows_names))
    print("Indices of selected rows regardless of filtering results: {}".format(slctd_rows))
    print('----------------------------------------------')
    print("Indices of all rows pre or post filerting: {}".format(order_of_rows_indices))
    print("Names of all rows pre or post filtering: {}".format(order_of_rows_names))
    print("----------------------------------------------")
    print("Complete data of active cell: {}".format(actv_cell))
    print("Complete data of all selected cells: {}".format(slctd_cell))
    
    #might need to change variable name due to conflicts
    dff = pd.DataFrame(all_rows_data)
    
    #highlight selected sections of bar chart
    colors = ['#7FDBFF' if i in slctd_row_indices else '#0074D9'
              for i in range(len(dff))]
    
    if "animal_type" in dff and "outcome_type" in dff:
        return [
            dcc.Graph(id='bar-chart',
                     figure=px.bar(
                         data_frame=dff,
                         x="animal_type",
                         y='outcome_type',
                         labels={"outcome_type": "% of animals with certain outcome type"}
                     ).update_layout(showlegend=False, xaxis={'categoryorder': 'total ascending'})
                     .update_traces(marker_color=colors, hovertemplate="<b>%{y}%</b><extra></extra>")
                     )
        ]

#----------------------------------------------------------------------------------


#----------------------------------------------------------------------------------
#MAP CHART CALLBACK
@app.callback(
    Output(component_id='choromap-container', component_property='children'),
    [Input(component_id='datatable-interactivity', component_property="derived_virtual_data"),
     Input(component_id='datatable-interactivity', component_property='derived_virtual_selected_rows')]
)

def update_map_new(all_rows_data, slctd_row_indices):
    dff=pd.DataFrame(all_row_data)
    
    # highlight selections
    borders = [5 if i in slctd_row_indices else 1
               for i in range(len(dff))]
    
    if "animal_type" in dff and "breed" in dff:
        return [
            dcc.Graph(id='choropleth',
                      style={'height': 700},
                      figure=px.choropleth(
                          data_frame = dff,
                          scope="usa",
                          color="internet daily",
                          title="% animal types and breeds with outcomes",
                          template='plotly_dark',
                          hover_data=['animal_type', 'breed'],
                      ).update_layout(showlegend=False, title=dict(font=dict(size=28), x=0.5, xanchor = 'center'))
                      .update_traces(marker_line_width=borders, hovertemplate="<b>%{customdata[0]}</b><br><br>" + "%{customdata[1]}" + "%")
                     )
        ]

#----------------------------------------------------------------------------------
    
    
    
# DOESN'T WORK __ Causes the entire program to hang. Attempted to fix it multiple times. continues to hang program

# @app.callback(
#    Output('map-id', "children"),
#    [Input('datatable-id', "derived_viewport_data")])

# def update_map(viewData,rows):
# #FIXME: Add in the code for your geolocation chart
# #If you completed the Module Six Assignment, you can copy in the code you created here.
#     dff = pd.DataFrame.from_dict(viewData)
#     #dff = viewDF.loc[rows]
#    #Austin TX is at [30.75,-97.48]
#     return [
#        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
#            dl.TileLayer(id="base-layer-id"),
#            # Marker with tool tip and popup
#            dl.Marker(position=[30.75,-97.48], children=[
#                dl.Tooltip(dff.iloc[0,4]),
#                dl.Popup([
#                    html.H1("Animal Name"),
#                    html.P(dff.loc[1,9])
#                ])
#            ])
#        ])
#    ]



app